In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob

In [ ]:
from google.colab import files
uploaded = files.upload()

importing dataset

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d moltean/fruits

In [ ]:
!unzip fruits.zip

In [ ]:
# loading the directories
training_dir = '/content/fruits-360_dataset/fruits-360/Training'
validation_dir = '/content/fruits-360_dataset/fruits-360/Test'
test_dir = '/content/fruits-360_dataset/fruits-360/test-multiple_fruits'

In [ ]:
# getting number of files
image_files = glob(training_dir + '/*/*.jp*g')
valid_image_files = glob(validation_dir + '/*/*.jp*g')

In [ ]:
test_files = glob(test_dir + '/*.jp*g')

In [ ]:
# getting the number of classes(types of fruits)
folders = glob(training_dir + '/*')
num_classes = len(folders)
print ('Total Classes = ' + str(num_classes))

Total Classes = 131


In [ ]:
# copying the pre-trained weights to our kernel
!mkdir ~/.keras
!mkdir ~/.keras/models
!cp ../input/keras-pretrained-models/*notop* ~/.keras/models/
!cp ../input/keras-pretrained-models/imagenet_class_index.json ~/.keras/models/

#Model

In [ ]:
from keras.models import Model
from keras.layers import Flatten, Dense
from keras.applications import VGG16

IMAGE_SIZE = [64, 64]

# loading the weights of VGG16
vgg = VGG16(input_shape = IMAGE_SIZE + [3], weights = 'imagenet', include_top = False)

for layer in vgg.layers:
    layer.trainable = False

x = Flatten()(vgg.output)
x = Dense(num_classes, activation = 'softmax')(x)

model = Model(inputs = vgg.input, outputs = x)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 64, 64, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 64, 64, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 32, 32, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 32, 32, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 32, 32, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 16, 16, 128)       0     

Train, test data generation

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input

training_datagen = ImageDataGenerator(
                                    rescale=1./255,   # all pixel values will be between 0 an 1
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    preprocessing_function=preprocess_input)

validation_datagen = ImageDataGenerator(rescale = 1./255, preprocessing_function=preprocess_input)

training_generator = training_datagen.flow_from_directory(training_dir, target_size = IMAGE_SIZE, batch_size = 200, class_mode = 'categorical')
validation_generator = validation_datagen.flow_from_directory(validation_dir, target_size = IMAGE_SIZE, batch_size = 200, class_mode = 'categorical')

Found 67692 images belonging to 131 classes.
Found 22688 images belonging to 131 classes.


In [ ]:
# checking the labels
training_generator.class_indices

Training

In [ ]:
training_images = 37836
validation_images = 12709

history = model.fit_generator(training_generator,
                   epochs = 1,
                   validation_data = validation_generator
                   )

<ipython-input-13-b930add88be4>:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(training_generator,


339/339 [==============================] - 3943s 12s/step - loss: 1.5133 - accuracy: 0.7549 - val_loss: 0.9250 - val_accuracy: 0.8293


Prediction

In [ ]:
print ('Training Accuracy = ' + str(history.history['accuracy']))
print ('Validation Accuracy = ' + str(history.history['val_accuracy']))

Training Accuracy = [0.7548750042915344]
Validation Accuracy = [0.8292930126190186]


In [ ]:
# saving the model
import pickle

with open('vgg16.pkl', 'wb') as file:
    pickle.dump(model, file)

In [ ]:
# loading the model
import pickle

pkl_file_path = '/content/vgg16.pkl'
with open(pkl_file_path, 'rb') as file:
    model = pickle.load(file)


In [ ]:
all_weights = []
for layer in model.layers:
    if hasattr(layer, 'get_weights'):
        weights = layer.get_weights()
        all_weights.append(weights)

for i, weights in enumerate(all_weights):
    print(f"Layer {i + 1} - {model.layers[i].name}")
    for j, w in enumerate(weights):
        print(f"   Weights {j + 1}: {w.shape}")

Layer 1 - input_1
Layer 2 - block1_conv1
   Weights 1: (3, 3, 3, 64)
   Weights 2: (64,)
Layer 3 - block1_conv2
   Weights 1: (3, 3, 64, 64)
   Weights 2: (64,)
Layer 4 - block1_pool
Layer 5 - block2_conv1
   Weights 1: (3, 3, 64, 128)
   Weights 2: (128,)
Layer 6 - block2_conv2
   Weights 1: (3, 3, 128, 128)
   Weights 2: (128,)
Layer 7 - block2_pool
Layer 8 - block3_conv1
   Weights 1: (3, 3, 128, 256)
   Weights 2: (256,)
Layer 9 - block3_conv2
   Weights 1: (3, 3, 256, 256)
   Weights 2: (256,)
Layer 10 - block3_conv3
   Weights 1: (3, 3, 256, 256)
   Weights 2: (256,)
Layer 11 - block3_pool
Layer 12 - block4_conv1
   Weights 1: (3, 3, 256, 512)
   Weights 2: (512,)
Layer 13 - block4_conv2
   Weights 1: (3, 3, 512, 512)
   Weights 2: (512,)
Layer 14 - block4_conv3
   Weights 1: (3, 3, 512, 512)
   Weights 2: (512,)
Layer 15 - block4_pool
Layer 16 - block5_conv1
   Weights 1: (3, 3, 512, 512)
   Weights 2: (512,)
Layer 17 - block5_conv2
   Weights 1: (3, 3, 512, 512)
   Weights 2: (

extracting weights, biases of each layer

In [ ]:
conv1_weights, conv1_biases = model.layers[1].get_weights()
conv2_weights, conv2_biases = model.layers[2].get_weights()
conv3_weights, conv3_biases = model.layers[4].get_weights()
conv4_weights, conv4_biases = model.layers[5].get_weights()
conv5_weights, conv5_biases = model.layers[7].get_weights()
conv6_weights, conv6_biases = model.layers[8].get_weights()
conv7_weights, conv7_biases = model.layers[9].get_weights()
conv8_weights, conv8_biases = model.layers[11].get_weights()
conv9_weights, conv9_biases = model.layers[12].get_weights()
conv10_weights, conv10_biases = model.layers[13].get_weights()
conv11_weights, conv11_biases = model.layers[15].get_weights()
conv12_weights, conv12_biases = model.layers[16].get_weights()
conv13_weights, conv13_biases = model.layers[17].get_weights()
dense1_weights, dense1_biases = model.layers[20].get_weights()

In [ ]:
dense1_weights_flattened_list = dense1_weights.flatten().tolist()

In [ ]:
# Converting biases to lists
conv1_biases_list = conv1_biases.tolist()
conv2_biases_list = conv2_biases.tolist()
conv3_biases_list = conv3_biases.tolist()
conv4_biases_list = conv4_biases.tolist()
conv5_biases_list = conv5_biases.tolist()
conv6_biases_list = conv6_biases.tolist()
conv7_biases_list = conv7_biases.tolist()
conv8_biases_list = conv8_biases.tolist()
conv9_biases_list = conv9_biases.tolist()
conv10_biases_list = conv10_biases.tolist()
conv11_biases_list = conv11_biases.tolist()
conv12_biases_list = conv12_biases.tolist()
conv13_biases_list = conv13_biases.tolist()
dense1_biases_list = dense1_biases.tolist()

Quantization of weights

In [ ]:
import torch

def absmax_quantize_list(weights_list):
    X = torch.tensor(weights_list, dtype=torch.float32)
    scale = 127 / torch.max(torch.abs(X))

    # Quantize
    X_quant = (scale * X).round()

    # Dequantize
    X_dequant = X_quant / scale

    quantized_list = X_quant.to(torch.int8).tolist()
    return quantized_list, X_dequant.tolist()

In [ ]:
new_conv_weights = []
for num_fil in range(0,512):
  for dep in range(0,512):
    for row in range(0,3):
      for col in range(0,3):
        new_conv_weights.append(conv13_weights[row][col][dep][num_fil])

In [ ]:
weights_list = new_conv_weights
print(len(weights_list))

quantized_weights, dequantized_weights = absmax_quantize_list(weights_list)

print("\nAbsmax quantized weights:")
print(len(quantized_weights))

with open('quantized_conv13_weights.txt', 'w') as f:
  f.write(str(quantized_weights))

2359296

Absmax quantized weights:
2359296


Testing

In [ ]:
from PIL import Image

image_path = '/content/fruits-360_dataset/fruits-360/Training/Dates/13_100.jpg'
new_width = 64
new_height = 64

image = Image.open(image_path)
resized_image = image.resize((new_width, new_height))
random_image = np.array(resized_image)

print(random_image)

[[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]]


In [ ]:
flattenedImage = []
for k in range(0, 3):
  for i in range(0, 64):
    for j in range(0, 64):
      flattenedImage.append(random_image[i][j][k])


In [ ]:
print(random_image.shape)

(64, 64, 3)


In [ ]:
with open('test_image.txt', 'w') as f:
  f.write(str(flattenedImage))

In [ ]:
# flattening the image
def flatten(image, h, w, d):
  flattenedImage = []
  for k in range(0, d):
    for i in range(0, h):
      for j in range(0, w):
        flattenedImage.append(image[i][j][k])
  return flattenedImage

taking some samples

In [ ]:
from PIL import Image
test_images = []
true_labels = []
tdata = []

for idx in range(0, 100):
    image = Image.open(test_files[idx])
    resized_image = image.resize((64, 64))
    reshaped_image = np.array(resized_image)
    # tdata.append(x_test[idx])
    test_images.append(flatten(reshaped_image, 64, 64, 3))
    # true_labels.append(y_test[idx])

flattened_test_images = [item for sublist in test_images for item in sublist]

In [ ]:
len(flattened_test_images)

1228800

In [ ]:
with open('test_images.txt', 'w') as f:
  f.write(str(flattened_test_images))

In [ ]:
import os
import time
from tensorflow.keras.preprocessing import image
image_filenames = test_files[0:100]
all_images = []
for filename in image_filenames:
    img = image.load_img(filename, target_size=(64, 64))
    img_array = image.img_to_array(img)
    img_array = img_array / 255.0
    all_images.append(img_array)


Time calculation

In [ ]:
all_images = np.array(all_images)
predictions = model.predict(all_images)

start_time = time.time()
predictions = model.predict(all_images)
end_time = time.time()
predicted_class_index = []
for i in range(len(predictions)):
  predicted_class_index.append(np.argmax(predictions[i]))
inference_time = []
inference_time.append(end_time - start_time)
print(f"Inference time: {inference_time[0]} seconds")

4/4 [==============================] - 5s 1s/step
Inference time: 5.170581817626953 seconds


In [ ]:
with open('predicted_labels.txt', 'w') as f:
  f.write(str(predicted_class_index))